In [1]:
import sys; sys.path.insert(0, '..') # So that we import the local copy of pyzx if you have installed from Github
import os
import csv

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

from multiprocessing import Pool
import pyzx as zx
from pyzx import cnot_mapper, architecture # Note that this is the local pyzx package from this repository/fork, not the one installed through pip or other means.
from pyzx import circuit
from pyzx import *
from pyzx.parity_maps import CNOT_tracker, build_random_parity_map
from pyzx.linalg import Mat2
from pyzx.simplify import full_reduce

Load Circuits for CombRowCol

In [2]:
comb_circuit_folder = "../circuits/combs-ZX/"

n_qubits = [9]
n_cnots = [30,40,50]
n_non_cnots = [0,1,2,4,5,8,10,15,20]
loaded_circuits = {}
for qubits in n_qubits:
    for cnots in n_cnots:
        for non_cnots in n_non_cnots:
            circuit_properties = {"Qubits"    : qubits,
                                  "CNOTs"     : cnots,
                                  "Non-CNOTs" : non_cnots}
            path = os.path.join(comb_circuit_folder,
                                str(circuit_properties["Qubits"])+"qubits", 
                                str(circuit_properties["CNOTs"])+"cnots", 
                                str(circuit_properties["Non-CNOTs"])+"non-cnots")
            loaded_circuits[f"({qubits},{cnots},{non_cnots})"] = []
            if os.path.exists(path):
                for file_name in os.listdir(path):
                    loaded_circuits[f"({qubits},{cnots},{non_cnots})"].append(circuit.Circuit.from_qasm_file(os.path.join(path, file_name)))
            else:
                print("Directory Does Not Exist")

In [3]:
from IPython.display import display, Markdown

DEBUG = False
OUTER_DISPLAY = False
INNTER_DISPLAY = False

for qubits in n_qubits:
    arch = architecture.create_architecture(architecture.SQUARE, n_qubits=qubits)
    with open(f'COMBROWCOL - {arch.name}', mode='w') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')

        for cnots in n_cnots:
            for non_cnots in n_non_cnots:
                equal = 0
                ratios = []
                for INDEX in range(100):
                    if False:
                        # Get circuit
                        circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                        # Draw circuit
                        OUTER_DISPLAY and display(zx.draw(circ))
                        # Decompose circuit into comb and holes
                        decomposition = CombDecomposition.from_circuit(circ.copy())
                        comb = decomposition.comb
                        # Draw comb
                        OUTER_DISPLAY and display(zx.draw(comb))
                        # Recompose circuit from comb and holes
                        new_circuit = CombDecomposition.to_circuit(decomposition)
                        # Display recomposed circuit
                        OUTER_DISPLAY and display(zx.draw(new_circuit))
                        # Check whether the recomposed circuit is the same as the original
                        equality = new_circuit.verify_equality(circ)
                        equal += equality
                    else:
                        circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                        new_circ, Id = combrowcol(circ, arch, DEBUG, OUTER_DISPLAY, INNTER_DISPLAY)
                        gate_ratio = (len(new_circ.gates) - non_cnots)/(len(circ.gates) - non_cnots) - 1
                        ratios.append(gate_ratio)
                        equality = new_circ.verify_equality(circ)
                        equal += equality
                        #print(f"INDEX: {INDEX} , GATE RATIO : {gate_ratio}")
                        #print(f"INDEX: {INDEX} {equality}")
                        #equal += Id
                        #print(f"Reduced to identity : {INDEX} {Id}")
                print(f"Qubits : {qubits}, CNOTs : {cnots}, Non-CNOTs : {non_cnots} -> {equal}, Ratio : {sum(ratios)/len(ratios)}")
                writer.writerow([qubits, cnots, non_cnots, sum(ratios)/len(ratios)])

Qubits : 9, CNOTs : 30, Non-CNOTs : 0 -> 100, Ratio : 0.9193333333333333
Qubits : 9, CNOTs : 30, Non-CNOTs : 1 -> 100, Ratio : 1.0803333333333331
Qubits : 9, CNOTs : 30, Non-CNOTs : 2 -> 100, Ratio : 1.1923333333333332
Qubits : 9, CNOTs : 30, Non-CNOTs : 4 -> 100, Ratio : 1.3676666666666668
Qubits : 9, CNOTs : 30, Non-CNOTs : 5 -> 100, Ratio : 1.5439999999999998
Qubits : 9, CNOTs : 30, Non-CNOTs : 8 -> 100, Ratio : 1.9106666666666674
Qubits : 9, CNOTs : 30, Non-CNOTs : 10 -> 100, Ratio : 2.145666666666666
Qubits : 9, CNOTs : 30, Non-CNOTs : 15 -> 100, Ratio : 2.4559999999999995
Qubits : 9, CNOTs : 30, Non-CNOTs : 20 -> 100, Ratio : 2.7643333333333335
Qubits : 9, CNOTs : 40, Non-CNOTs : 0 -> 100, Ratio : 0.5312500000000001
Qubits : 9, CNOTs : 40, Non-CNOTs : 1 -> 100, Ratio : 0.628
Qubits : 9, CNOTs : 40, Non-CNOTs : 2 -> 100, Ratio : 0.7892500000000002
Qubits : 9, CNOTs : 40, Non-CNOTs : 4 -> 100, Ratio : 0.9649999999999999
Qubits : 9, CNOTs : 40, Non-CNOTs : 5 -> 100, Ratio : 1.168249

Cut and Route Method

In [4]:
def subcircuits(circuit):
    sub_circuits = []
    for gate in circuit.gates:
        if len(sub_circuits) == 0:
            sub_circuits.append([gate])
        else:
            current_name = sub_circuits[-1][0].name
            if current_name == "CNOT":
                if gate.name == "CNOT":
                    sub_circuits[-1].append(gate)
                else:
                    sub_circuits.append([gate])
            else:
                if gate.name == "CNOT":
                    sub_circuits.append([gate])
                else:
                    sub_circuits[-1].append(gate)
    return sub_circuits

def cutrowcol(circuit, arch):
    sub_circuits = subcircuits(circ.copy())
    new_circuit = Circuit(circuit.qubits)
    for sub_circuit in sub_circuits:
        if sub_circuit[0].name != "CNOT":
            new_circuit.gates = new_circuit.gates + sub_circuit
        else:
            # Get matrix corresponding to CNOT sub circuit
            cnottracker = CNOT_tracker(circuit.qubits)
            cnottracker.gates = sub_circuit
            cnottracker.update_matrix()
            temp_circuit = CNOT_tracker(circuit.qubits)
            # rowcol reduced the matrix of cnottracker to the identity under the constraints of
            # arch, logging each row operation at a CNOT in temp_circuit
            rowcol(cnottracker.matrix, arch, temp_circuit, debug=None)
            # The gates of temp_circuit are then added to the new_circuit
            new_circuit.gates = new_circuit.gates + temp_circuit.gates
    return new_circuit
    

In [5]:
from IPython.display import display, Markdown

DEBUG = False
OUTER_DISPLAY = False
INNTER_DISPLAY = False


for qubits in n_qubits:
    arch = architecture.create_architecture(architecture.SQUARE, n_qubits=qubits)
    with open(f'CUTROWCOL - {arch.name}', mode='w') as csv_file:
        #fieldnames = ['Number of Qubits', 'Number of CNOT Gates', 'Number of Non CNOT Gates']
        writer = csv.writer(csv_file, delimiter=',')

        for cnots in n_cnots:
            for non_cnots in n_non_cnots:
                equal = 0
                ratios = []
                for INDEX in range(100):
                    if False:
                        circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]
                        print(f"Gates : {circ.gates}")
                        print(f"Sub Circuits : {subcircuits(circ)}")
                    else:
                        circ = loaded_circuits[f"({qubits},{cnots},{non_cnots})"][INDEX]                                        
                        new_circ = cutrowcol(circ, arch)
                        gate_ratio = (len(new_circ.gates) - non_cnots)/(len(circ.gates) - non_cnots) - 1
                        ratios.append(gate_ratio)
                        equality = new_circ.verify_equality(circ)
                        equal += equality
                        #print(f"INDEX: {INDEX} , GATE RATIO : {gate_ratio}")
                        #print(f"INDEX: {INDEX} {equality}")
                print(f"Qubits : {qubits}, CNOTs : {cnots}, Non-CNOTs : {non_cnots} -> {equal}, Ratio : {sum(ratios)/len(ratios)}")
                writer.writerow([qubits, cnots, non_cnots, sum(ratios)/len(ratios)])

Qubits : 9, CNOTs : 30, Non-CNOTs : 0 -> 100, Ratio : 0.7453333333333331
Qubits : 9, CNOTs : 30, Non-CNOTs : 1 -> 100, Ratio : 1.395
Qubits : 9, CNOTs : 30, Non-CNOTs : 2 -> 100, Ratio : 1.8813333333333337
Qubits : 9, CNOTs : 30, Non-CNOTs : 4 -> 100, Ratio : 2.2833333333333328
Qubits : 9, CNOTs : 30, Non-CNOTs : 5 -> 100, Ratio : 2.4363333333333332
Qubits : 9, CNOTs : 30, Non-CNOTs : 8 -> 100, Ratio : 2.862999999999999
Qubits : 9, CNOTs : 30, Non-CNOTs : 10 -> 100, Ratio : 2.9586666666666654
Qubits : 9, CNOTs : 30, Non-CNOTs : 15 -> 100, Ratio : 3.1189999999999993
Qubits : 9, CNOTs : 30, Non-CNOTs : 20 -> 100, Ratio : 3.2483333333333335
Qubits : 9, CNOTs : 40, Non-CNOTs : 0 -> 100, Ratio : 0.38325
Qubits : 9, CNOTs : 40, Non-CNOTs : 1 -> 100, Ratio : 1.0415
Qubits : 9, CNOTs : 40, Non-CNOTs : 2 -> 100, Ratio : 1.43925
Qubits : 9, CNOTs : 40, Non-CNOTs : 4 -> 100, Ratio : 2.0165000000000006
Qubits : 9, CNOTs : 40, Non-CNOTs : 5 -> 100, Ratio : 2.253499999999999
Qubits : 9, CNOTs : 40, 